### Custom Shallow Network - miniVGG 
With batch normalization and (optional) random cropping of the image

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [ ]:
import os
import numpy as np
from glob import glob
from tqdm import tqdm
import datetime
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt 

%matplotlib inline

In [ ]:
# dimensions of our images.
img_width, img_height = 224, 224
train_data_dir = '/home/sanchit/Documents/Projects/datasets/fire_and_smoke_data/train/'
validation_data_dir = '/home/sanchit/Documents/Projects/datasets/fire_and_smoke_data/val/'
nb_train_samples = 2400
nb_validation_samples = 490
epochs = 100
batch_size = 32
model_path = "./models/cust_network.h5"
init_lr = 0.003

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
model = Sequential()
input_shape = (img_width, img_height, 3)
model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='he_normal', 
                 kernel_regularizer=l2(1e-4), input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3), padding='same', strides=2, kernel_initializer='he_normal', 
                 kernel_regularizer=l2(1e-4), input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='he_normal', 
                 kernel_regularizer=l2(1e-4), input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3), padding='same', strides=2, kernel_initializer='he_normal', 
                 kernel_regularizer=l2(1e-4), input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='he_normal', 
                 kernel_regularizer=l2(1e-4), input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3), padding='same', strides=2, kernel_initializer='he_normal', 
                 kernel_regularizer=l2(1e-4), input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal', 
                 kernel_regularizer=l2(1e-4), input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3), padding='same', strides=2, kernel_initializer='he_normal', 
                 kernel_regularizer=l2(1e-4), input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(128, (3, 3), padding='same', kernel_initializer='he_normal', 
                 kernel_regularizer=l2(1e-4), input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(128, (3, 3), padding='same', strides=2, kernel_initializer='he_normal', 
                 kernel_regularizer=l2(1e-4), input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(GlobalAveragePooling2D())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

In [ ]:
#opt = SGD(lr=init_lr, momentum=0.9) # 93.33%
#opt = SGD(lr=init_lr, momentum=0.9, decay=init_lr) # 88.96 %
#opt = Adam() # 94.17 %
#opt = RMSprop() #  %
opt = SGD()
#opt = Adam(learning_rate=init_lr) # 93.12 %

model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
def load_data(dir_path="", class_mode='binary', classes=None):
    """
    loads all the images for all the classes (sub-dirs) provided in the input directory.
    :param dir_path -  input directory which contains for each class a sub-directory. 
    :param class_mode - binary (for usage in binary_crossentropy loss) and categorical (categorical_crossentropy loss)
    :params classes - a list of classes'names 
    """
    X = []
    y = []
    data_kind = dir_path.split("/")[-2] # either training or validation or testing
    directories=[d for d in os.listdir(dir_path) if os.path.isdir(d) or (not d.startswith("."))]
    
    for label, class_name in enumerate(directories):
        print(f"loading {data_kind} data for class: {class_name}")
        class_dir = os.path.join(dir_path, class_name)
        for img_path in tqdm(glob(class_dir + '/*.jpg')):
            img = load_img(img_path, target_size=(img_width, img_height))
            img = img_to_array(img)

            # save the image and its corresponding label
            X.append(img)
            y.append(label)
                
    X = np.asarray(X, dtype=np.float32) / 255.0
    y = np.asarray(y)
    
    if class_mode == "categorical":
        y = to_categorical(y_train, num_classes=classes)
        
    print(f"total number of images loaded: {X.shape[0]} and of shape: {X.shape[1:]}")
    
    return X, y

In [ ]:
X_train, y_train = load_data(dir_path=train_data_dir)
X_val, y_val = load_data(dir_path=validation_data_dir)

In [ ]:
# create a train generator
train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True, 
    vertical_flip=True,
    rotation_range=30,
    fill_mode="wrap",
    height_shift_range=0.15,
    width_shift_range=0.15)

# create a test/val generator
test_datagen = ImageDataGenerator()

In [ ]:
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size, shuffle=True)
validation_generator = test_datagen.flow(X_val, y_val, batch_size=batch_size, shuffle=False)

In [ ]:
# define callbacks before starting the training
# early_stop = EarlyStopping(monitor="val_loss", patience=10, mode="min", verbose=1)
# model_checkpoint = ModelCheckpoint(model_path, monitor="val_accuracy", save_best_only=True, mode='max', verbose=1)
# logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
# tensorboard = TensorBoard(log_dir=logdir, histogram_freq=1)
# callbacks = [early_stop, model_checkpoint, tensorboard]

H = model.fit_generator(train_generator, 
                        steps_per_epoch=nb_train_samples // batch_size, 
                        epochs=epochs,
                        validation_data=validation_generator,
                        validation_steps=nb_validation_samples // batch_size, 
                        workers = 8)

# launch TensorBoard
#%tensorboard --logdir logs

In [ ]:
N = np.arange(0, epochs)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.show()